# <center> Clients for Enhanced Tornado Server Example
## <center> ENGR 580A2: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="600" /> 
### <center> Instructor: Dr. Jeremy Daily<br>
### <center> Content Author: Jerry Duggan<br>Fall 2020

# Performance of Symmetric v. Asymmetric Encryption

See file EncryptionTiming.py

* INFO:Timer:Symmetric Encryptions: 1000, secs: 0.116792, Symmetric Encryptions per second: 8562.2
* INFO:Timer:Asymmetric Encryptions: 100, secs: 0.46274, Asymmetric Encryptions per second: 216.1
* INFO:Timer:Asymmetric Signing: 100, secs: 0.447826, Asymmetric Signing per second: 223.3

# SimpleClient
Basic functionality -- accept a POST request, and return the original "cipher_text" (which is not encrypted) as "plaintext"

In [1]:
import requests

plain_text = "I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character."


r = requests.post("http://localhost:9100/",
                      json={'cipher_text': plain_text})
reply = r.json()
print(reply['plaintext'])

I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character.


# SymmetricKeyClient
Server will respond to a GET request with a symmetric key.  Client then uses this key to encrypt
a message and POST it to the server.  Server decrypts the message with the symmetric key and returns
cipher_text and plaintext back to the client.

**VERY BAD PRACTICE** as symmetric key can be snooped.  No security here at all.

In [21]:
import requests
from cryptography.fernet import Fernet
# Get the key (Don't actually do this)
r = requests.get("http://localhost:9100/encrypted/")
key = r.json()['key']
print(key)
f = Fernet(key)
plain_text = "I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character."
cipher_text = f.encrypt(plain_text.encode('utf-8'))
r = requests.post("http://localhost:9100/encrypted/", json={'cipher_text': cipher_text.decode('utf-8'), })
reply = r.json()
print(reply['plaintext'])

Z6T8Z93EGDRWLuFKAqEpTh6PZ1SKHKygO22wVTeGIzQ=
I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character.


# PublicKeyClient
Server responds to a GET request with its public key.  Client then uses this public key to encrypt a message.  Server uses its private key to decrypt, returning both the original cipher_text and the decrypted plaintext.

This is reasonable practice, as only the server knows its private key, and thus only the server can decrypt, so message privace is maintained.  It is subject to impersonation of the client, though...

It is also potentially slow, as asymmetric encryption is significantly slower than symmetric encryption.

![Public Key](PublicKey.png)

In [13]:
import requests
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
import base64

r = requests.get("http://localhost:9100/serverPublicKey/")
jsonReply = r.json()
serverPublicKeyPEM = jsonReply['publicKey'].encode('ASCII')
serverPublicKey = serialization.load_pem_public_key(serverPublicKeyPEM, backend=default_backend())

plain_text = "I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character."
encodedPlainText = plain_text.encode('utf-8')
encrypted = serverPublicKey.encrypt(encodedPlainText,
                                    padding.OAEP(
                                        mgf=padding.MGF1(algorithm=hashes.SHA256()),
                                        algorithm=hashes.SHA256(),
                                        label=None)
                                    )

encryptedB64 = base64.b64encode(encrypted)
encryptedB64ASCII = encryptedB64.decode('ASCII')

r = requests.post("http://localhost:9100/serverPublicKey/",
                  json={'cipher_text': encryptedB64ASCII})
reply = r.json()
print(reply['plaintext'])




I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character.


# KeyExchangeClient
Client sends its public key as a URL argument in the GET.  Server replies to the get by encrypting a symmetric key with the client public key.  Client uses its private key to decrypt the symmetric key, and sends subsequent messages encryped with the symmetric key.

Much faster than using asymmetric encryption for the entire conversation.

Still subject to impersonation -- this time on the server side.  Can be cracked by a Person-in-the-middle (PITM) attack.

![Key Exchange](KeyExchange.png)

In [24]:
import requests
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from cryptography.fernet import Fernet

import base64

PRIVATE_KEY = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
PUBLIC_KEY = PRIVATE_KEY.public_key()
PORT = 9100  # set this here so we can reuse app for PITM

# Prepare the client's public key for transmission to server & transmit as a URL parameter

encodedClientPublicKey = base64.b64encode(PUBLIC_KEY.public_bytes(Encoding.DER, 
                                                                  PublicFormat.SubjectPublicKeyInfo))
r = requests.get(f"http://localhost:{PORT}/serverPublicKeySymmetricKeyExchange/", 
                 {'client_publicKey': encodedClientPublicKey})
jsonReply = r.json()

# The symmetric key will be returned (encoded) in the 'encocedEncryptedSymmetricKey' parameter
# from the server.  Prepare it for use by decoding then decrypting using the client private key

encodedEncryptedSymmetricKey = jsonReply['encodedEncryptedSymmetricKey']
encryptedSymmetricKey = base64.b64decode(encodedEncryptedSymmetricKey)
symmetricKey = PRIVATE_KEY.decrypt(encryptedSymmetricKey, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),
                                                                      algorithm=hashes.SHA256(),
                                                                      label=None))
print(symmetricKey)

# Now we can send our message using the symmetric key -- just like the SymmetricKeyClient example
# (except this time the symmetric key cannot be snooped)

f = Fernet(symmetricKey)
plain_text = "I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character."
cipher_text = f.encrypt(plain_text.encode('utf-8'))
r = requests.post(f"http://localhost:{PORT}/encrypted/", json={'cipher_text': cipher_text.decode('utf-8'), })
reply = r.json()
print(reply['plaintext'])

b'Z6T8Z93EGDRWLuFKAqEpTh6PZ1SKHKygO22wVTeGIzQ='
I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character.


# Person in the middle (PITM) attach
1. Start the PITM.py in a separate window
1. Change the port in the above KeyExchangeClient to 9101
1. Rerun the KeyExchangeClient

![Person in the Middle](PITM.png)

## Certificates and Chain-of-Trust combats PITM attacks
If I trust you, and you trust the server, then I can trust the server.

![Certificate Creation](CA.png)

### Who do I trust?
Windows certificate manger establishes a trust store for windows apps.  Provides public keys for trusted Certificate Authorities (CA)s.

1. Windows Start->Run
2. Enter mmc
3. File->Add or Remove Snap-in...
4. Select 'Certificates' & 'Add>'
5. Add for computer account
6. Finish, then OK